<a href="https://colab.research.google.com/github/AleAguirreM/Analitica3AA/blob/main/e_despliegue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import openpyxl
####Paquete para sistema basado en contenido ####
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
import sys
import codecs

In [53]:
#conectamos drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
sys.path.append('/content/drive/MyDrive')

In [55]:
import a_funciones as fn

In [68]:
def preprocesar():

    #### conectar_base_de_Datos#################
    conn=sql.connect('/content/drive/MyDrive/db_moviesSQL')
    cur=conn.cursor()


    ######## convertir datos crudos a bases filtradas por usuarios que tengan cierto número de calificaciones
    fn.ejecutar_sql('/content/drive/MyDrive/prepoce.sql', cur)

    ##### llevar datos que cambian constantemente a python ######
    movies=pd.read_sql('select * from full', conn )
    ratings=pd.read_sql('select * from ratings_final', conn)
    usuarios=pd.read_sql('select distinct (userId) as user_id from full',conn)


    #### transformación de datos crudos - Preprocesamiento ################
    movies['year']=movies.year.astype('int')

    ##### escalar para que año esté en el mismo rango ###
    sc=MinMaxScaler()
    movies[["year_sc"]]=sc.fit_transform(movies[['year']])

    ## eliminar filas que no se van a utilizar ###
    movies_dum1= movies.drop(columns=['movieId','title','cnt_rat','year','date','userId','date','index'])

    return movies_dum1,movies, conn, cur

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
##########################################################################
###############Función para entrenar modelo por cada usuario ##########
###############Basado en contenido todo lo visto por el usuario Knn#############################
def recomendar(user_id):

    movies_dum1, movies, conn, cur= preprocesar()

    ratings=pd.read_sql('select * from full where userId=:user',conn, params={'user':user_id})
    l_movies_r=ratings['movieId'].to_numpy()
    movies_dum1[['movieId','title']]=movies[['movieId','title']]
    movies_r=movies_dum1[movies_dum1['movieId'].isin(l_movies_r)]
    movies_r=movies_r.drop(columns=['movieId','title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    centroide=movies_r.groupby("indice").mean()


    movies_nr=movies_dum1[~movies_dum1['movieId'].isin(l_movies_r)]
    movies_nr=movies_nr.drop(columns=['movieId','title'])
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0]
    recomend_b=movies.loc[ids][['title','movieId']]


    return recomend_b



In [75]:
def main(list_user):
    recomendaciones_todos = pd.DataFrame()
    for user_id in list_user:
        recomendaciones = recomendar(user_id)
        if recomendaciones is not None:
            recomendaciones["user_id"] = user_id
            recomendaciones.reset_index(inplace=True, drop=True)
            recomendaciones_todos = pd.concat([recomendaciones_todos, recomendaciones])

    recomendaciones_todos.to_excel('C:\\cod\\Analitica3AA\\salidas\\reco\\recomendaciones.xlsx')
    recomendaciones_todos.to_csv('C:\\cod\\Analitica3AA\\salidas\\reco\\recomendaciones.csv')

if __name__ == "__main__":
    list_user = [1, 2, 3, 4]
    main(list_user)